In [ ]:
!nvidia-smi

In [ ]:
!pip install numpy
!pip install torch torchvision torchaudio
!pip install datasets datasets[audio] datasets[audio]
!pip install deepspeed
!pip install accelerate
!pip install transformers
!pip install datasets
!pip install ftfy
!pip install tensorboard
!pip install Jinja2

In [ ]:
!pip install xformers

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git@9baa29e9c047c64756f2d4b9e542677eb8833270
!git clone https://github.com/huggingface/diffusers.git
!cd diffusers/ && git checkout 9baa29e9c047c64756f2d4b9e542677eb8833270

In [ ]:
model_name = "SWFPetite"
version = "2"
output_dir = f"/content/drive/MyDrive/RawData/{model_name}-{version}"
data_dir = f"/content/drive/MyDrive/RawData/data/out/{model_name}"
cache_dir = f"/content/drive/MyDrive/RawData/{model_name}"
pretrained_model_name_or_path= f"/content/SWFPetite-1"

In [ ]:
!mkdir -p $data_dir

In [ ]:
import pandas as pd
from PIL import Image
from io import BytesIO
import os
df = pd.read_parquet(f"/content/drive/MyDrive/RawData/data/out/{model_name}.parquet")

training_lines = []

for record in df.to_dict(orient='records'):
    image_name = record['image_name']
    caption = record['caption']
    text = record['title']
    out_record = { "file_name" : image_name, "text": [text, caption] }
    image_path = f"{data_dir}/{image_name}"
    if os.path.exists(image_path):
        training_lines.append(out_record)
        continue
    else:
        img = Image.open(BytesIO(record["image_data"]))
        img.save(image_path)
        img.close()
        if os.path.exists(image_path):
            training_lines.append(out_record)
        else:
            continue
final = pd.DataFrame(data=training_lines)
final.to_json(f"{data_dir}/metadata.jsonl", orient="records", lines=True)
display(final)

In [ ]:
!ls $data_dir | wc -l
!cat $data_dir/metadata.jsonl | wc -l

In [ ]:
things = []
extant_files = os.listdir(data_dir)
for f in extant_files:
    a = final.loc[final['file_name'].isin([f])]
    text = a['text']
    im = a['file_name']
    try:
        record = a.to_dict(orient='records')[0]
    except:
        continue
    things.append(record)
some_things = pd.DataFrame(data=things)

# sampled = some_things.sample(n=500)

some_things.to_json(f"{data_dir}/metadata.jsonl", orient="records", lines=True)

In [ ]:
!ls $data_dir | wc -l
!cat $data_dir/metadata.jsonl | wc -l

In [ ]:
!cp -r /content/drive/MyDrive/RawData/SWFPetite-1 /content/

In [ ]:
!accelerate launch --config_file /content/drive/MyDrive/RawData/default_config.yaml \
/content/diffusers/examples/text_to_image/train_text_to_image.py \
--pretrained_model_name_or_path=$pretrained_model_name_or_path \
--train_data_dir=$data_dir \
--use_ema \
--resolution=512 \
--center_crop --random_flip \
--num_train_epochs=100 \
--train_batch_size=16 \
--gradient_accumulation_steps=5 \
--gradient_checkpointing \
--mixed_precision="bf16" \
--learning_rate=1e-05 \
--cache_dir=$cache_dir \
--max_grad_norm=1 \
--lr_scheduler="constant" \
--lr_warmup_steps=1 \
--allow_tf32 \
--seed=1 \
--enable_xformers_memory_efficient_attention \
--output_dir=$output_dir

In [ ]:
from diffusers import DiffusionPipeline, EulerDiscreteScheduler, DPMSolverMultistepScheduler, StableDiffusionPipeline

stable_diffusion = DiffusionPipeline.from_pretrained("/content/drive/MyDrive/RawData/SWFPetite-2", safety_checker=None)

diff_pipe: DiffusionPipeline = DiffusionPipeline.from_pretrained("/content/drive/MyDrive/RawData/SWFPetite-2", safety_checker=None)

pipe = StableDiffusionPipeline(**diff_pipe.components)

pipe.to("cuda")
#
prompt = "A busty, gourgous"
#
image = pipe(prompt, height=512, width=512, guidance_scale=8, num_inference_steps=50).images[0]
#
display(image)

In [ ]:
from google.colab import runtime
runtime.unassign()